# NW GMSA

## pymupdf

In [33]:
import pymupdf # imports the pymupdf library

pdf_file = "PDF/R24-00BX-1.pdf"
doc = pymupdf.open(pdf_file)

html_text = ""
all_text = ""
for page in doc:  # iterate the document pages
    page.read_contents()
    html_text += page.get_text('html', clip = None)
    all_text += page.get_text('text', clip = None)

with open("nw-gmsa-pymupfd.html", "wb", ) as f:
    f.write(html_text.encode("utf-8"))
with open("nw-gmsa-pymupfd.txt", "wb", ) as f:
    f.write(all_text.encode("utf-8"))

## PyPDF2

In [34]:
from PyPDF2 import PdfReader

pdf_object = PdfReader(pdf_file)
all_text = ""
for page in pdf_object.pages:
    all_text += page.extract_text() + "\n"
with open("nw-gmsa-PyPDF2.txt", "wb", ) as f:
    f.write(all_text.encode("utf-8"))

## tabula

In [35]:
import tabula

dfs = tabula.read_pdf(pdf_file, pages="all", stream=True, encoding='cp1252',pandas_options={'header': None})
tabula.environment_info()
print(len(dfs))

Python version:
    3.13.3 (tags/v3.13.3:6280bb5, Apr  8 2025, 14:47:33) [MSC v.1943 64 bit (AMD64)]
Java version:
    openjdk version "21.0.1" 2023-10-17 LTS
OpenJDK Runtime Environment Microsoft-8526870 (build 21.0.1+12-LTS)
OpenJDK 64-Bit Server VM Microsoft-8526870 (build 21.0.1+12-LTS, mixed mode, sharing)
tabula-py version: 2.10.0
platform: Windows-11-10.0.26100-SP0
uname:
    uname_result(system='Windows', node='DESKTOP-4A7A8NJ', release='11', version='10.0.26100', machine='AMD64')
linux_distribution: ('', '', '')
mac_ver: ('', ('', '', ''), '')
2


In [36]:
dfs[0]

,0,1,2,3,4,5
0,Forename:,Geneticsthree,Report No:,R24-00BX-1,NaN,NaN
1,Surname:,BEAKER,Hospital No:,Not provided,NaN,NaN
2,Sex:,Male,Other No:,Not provided,NaN,NaN
3,DoB:,01/01/1970,Sample:,DNA (External),NaN,NaN
4,NHS No:,NaN,NaN,NaN,NaN,NaN
5,Hive Order ID:,1000104558,NaN,NaN,NaN,NaN
6,Collected:,Not provided,Received:,03/06/2024 12:12,Activated:,03/06/2024


In [37]:
dfs[1]

,0,1
0,HGVS description: NM_001320.6(CSNK2B):c.256C>T...,Classification: Likely pathogenic
1,Genomic coordinates: Chr6(GRCh38):g.31668619C>T,Zygosity: Heterozygous Inheritance: de novo


In [38]:
import json
observationJson = {
    "meta" : {
        "profile" : [
            "https://nw-gmsa.github.io/StructureDefinition/Observation-Variant"
        ]
    },
    "status" : "final",
    "category":[
        {
            "coding" : [
                {
                    "system" : "http://terminology.hl7.org/CodeSystem/observation-category",
                    "code" : "laboratory"
                }
            ]
        },
        {
            "coding" : [
                {
                    "system" : "http://terminology.hl7.org/CodeSystem/v2-0074",
                    "code" : "GE"
                }
            ]
        }
    ],
    "code" : {
        "coding" : [
            {
                "system" : "http://loinc.org",
                "code" : "69548-6",
                "display" : "Genetic variant assessment"
            }
        ]
    },
    "valueCodeableConcept" : {
        "coding" : [
            {
                "system" : "http://loinc.org",
                "code" : "LA9633-4",
                "display" : "Present"
            }
        ]
    },
    "subject" : {
        "reference" : "urn:uuid:d6faafcf-db64-4c11-9da8-25f36774c1bd",
        "identifier" : {
            "system" : "https://fhir.nhs.uk/Id/nhs-number",
            "value" : "9449305552"
        },
        "display" : "Octavia CHISLETT"
    },
    "method" : {
        "coding" : [
            {
                "system" : "http://loinc.org",
                "code" : "LA26398-0",
                "display" : "Sequencing"
            }
        ]
    },
    "component" : []
}

In [39]:
hgvs = str(dfs[1].loc[0, 0]).replace('HGVS description: ','')
print(hgvs)
genename = hgvs[:hgvs.index(":")]
#fix hard coding
hgnc = "HGNC:1100"
observationJson['component'].append( {
    "code" : {
        "coding" : [
            {
                "system" : "http://loinc.org",
                "code" : "48013-7"
            }
        ]
    },
    "valueCodeableConcept" : {
        "coding" : [
            {
                "system" : "http://www.genenames.org",
                "code" : hgnc,
                "display" : genename
            }
        ]
    }
})

observationJson['component'].append({
    "code" : {
        "coding" : [
            {
                "system" : "http://loinc.org",
                "code" : "48004-6",
                "display" : "DNA change"
            }
        ]
    },
    "valueCodeableConcept" : {
        "coding" : [
            {
                "system" : "http://varnomen.hgvs.org",
                "code" : "BRCA1:c.3143delG p.(Gly1048ValfsTer14)"
            }
        ]
    }
})

zygosity = str(dfs[1].loc[1, 1]).replace('Zygosity: ','')
zygosity = zygosity[:zygosity.index("Inheritance")]
print(zygosity)
# Need to remove hard coding on this code
code = "LA6706-1"
observationJson['component'].append({
    "code" : {
        "coding" : [
            {
                "system" : "http://loinc.org",
                "code" : "53034-5",
                "display" : "Allelic state"
            }
        ]
    },
    "valueCodeableConcept" : {
        "coding" : [
            {
                "system" : "http://loinc.org",
                "code" : code,
                "display" : zygosity
            }
        ]
    }
})


NM_001320.6(CSNK2B):c.256C>T p.(Arg86Cys)
Heterozygous 


In [40]:
print(json.dumps(observationJson, indent=2, ensure_ascii=False))

{
  "meta": {
    "profile": [
      "https://nw-gmsa.github.io/StructureDefinition/Observation-Variant"
    ]
  },
  "status": "final",
  "category": [
    {
      "coding": [
        {
          "system": "http://terminology.hl7.org/CodeSystem/observation-category",
          "code": "laboratory"
        }
      ]
    },
    {
      "coding": [
        {
          "system": "http://terminology.hl7.org/CodeSystem/v2-0074",
          "code": "GE"
        }
      ]
    }
  ],
  "code": {
    "coding": [
      {
        "system": "http://loinc.org",
        "code": "69548-6",
        "display": "Genetic variant assessment"
      }
    ]
  },
  "valueCodeableConcept": {
    "coding": [
      {
        "system": "http://loinc.org",
        "code": "LA9633-4",
        "display": "Present"
      }
    ]
  },
  "subject": {
    "reference": "urn:uuid:d6faafcf-db64-4c11-9da8-25f36774c1bd",
    "identifier": {
      "system": "https://fhir.nhs.uk/Id/nhs-number",
      "value": "9449305552"
    

In [41]:
from fhir.resources.observation import Observation
observation = Observation(**observationJson)

# can now post using FHIR python libraries